In [6]:
# Import the dependencies.
import pandas as pd
import gmaps
import requests
# Import the API key.
from config import g_key

In [7]:
# Store the CSV you saved created in part one into a DataFrame.
city_data_df = pd.read_csv("weather_data/cities.csv")
city_data_df.head()

,City_ID,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed
0,0,Sao Joao Da Barra,BR,2021-09-19 17:20:39,-21.6403,-41.0511,82.49,70,12,15.97
1,1,Bredasdorp,ZA,2021-09-19 17:20:27,-34.5322,20.0403,60.12,67,3,13.80
2,2,Beringovskiy,RU,2021-09-19 17:20:39,63.0500,179.3167,40.10,88,99,7.58
3,3,Maiduguri,NG,2021-09-19 17:19:08,11.8464,13.1603,89.04,44,74,3.91
4,4,Jamestown,US,2021-09-19 17:19:06,42.0970,-79.2353,73.02,53,1,4.00


In [8]:
city_data_df.dtypes

City_ID         int64
City           object
Country        object
Date           object
Lat           float64
Lng           float64
Max Temp      float64
Humidity        int64
Cloudiness      int64
Wind Speed    float64
dtype: object

In [25]:
# Configure gmaps to use your Google API key.
gmaps.configure(api_key=g_key)

In [26]:
# Heatmap of temperature
# Get the latitude and longitude.
locations = city_data_df[["Lat", "Lng"]]
# Get the maximum temperature.
# Get the maximum temperature.
max_temp = city_data_df["Max Temp"]
temps = []
for temp in max_temp:
    temps.append(max(temp, 0))
# Assign the figure variable.
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
# Assign the heatmap variable.
heat_layer = gmaps.heatmap_layer(locations, weights=[max(temp, 0) for temp in max_temp], dissipating=False, max_intensity=300, point_radius=4)
# Add the heatmap layer.
fig.add_layer(heat_layer)
# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

In [27]:
# Heatmap of percent humidity
locations = city_data_df[["Lat", "Lng"]]
humidity = city_data_df["Humidity"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, dissipating=False, max_intensity=300, point_radius=4)

fig.add_layer(heat_layer)
# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

In [1]:
# Ask the customer to add a minimum and maximum temperature value.
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 60
What is the maximum temperature you would like for your trip? 90


In [2]:
# Filter the dataset to find the cities that fit the criteria.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df

NameError: name 'city_data_df' is not defined

In [30]:
preferred_cities_df.count()

City_ID       424
City          424
Country       418
Date          424
Lat           424
Lng           424
Max Temp      424
Humidity      424
Cloudiness    424
Wind Speed    424
dtype: int64

In [35]:
preferred_cities_df = preferred_cities_df.dropna()

In [36]:
preferred_cities_df.count()

City_ID       418
City          418
Country       418
Date          418
Lat           418
Lng           418
Max Temp      418
Humidity      418
Cloudiness    418
Wind Speed    418
dtype: int64

In [38]:
# Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = preferred_cities_df[["City", "Country", "Max Temp", "Lat", "Lng"]].copy()
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Lat,Lng,Hotel Name
0,Sao Joao Da Barra,BR,82.49,-21.6403,-41.0511,
1,Bredasdorp,ZA,60.12,-34.5322,20.0403,
3,Maiduguri,NG,89.04,11.8464,13.1603,
4,Jamestown,US,73.02,42.0970,-79.2353,
5,Bengkulu,ID,74.84,-3.8004,102.2655,
6,Puerto Ayora,EC,78.76,-0.7393,-90.3518,
9,Saint-Philippe,RE,67.59,-21.3585,55.7679,
11,Luau,AO,83.16,-10.7073,22.2247,
12,Touros,BR,86.90,-5.1989,-35.4608,
13,Kruisfontein,ZA,58.89,-34.0033,24.7314,


In [39]:
# Set parameters to search for a hotel.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

In [44]:
# Iterate through the DataFrame.
for index, row in hotel_df.iterrows():
    # Get the latitude and longitude.
    lat = row["Lat"]
    lng = row["Lng"]

    # Add the latitude and longitude to location key for the params dictionary.
    params["location"] = f"{lat},{lng}"

    # Use the search term: "lodging" and our latitude and longitude.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    # Make request and get the JSON data from the search.
    hotels = requests.get(base_url, params=params).json()
    # Grab the first hotel from the results and store the name.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not foun

In [47]:
hotel_df.head(10)

,City,Country,Max Temp,Lat,Lng,Hotel Name
0,Sao Joao Da Barra,BR,82.49,-21.6403,-41.0511,Pousada Porto De Canoas
1,Bredasdorp,ZA,60.12,-34.5322,20.0403,Bredasdorp Country Manor
3,Maiduguri,NG,89.04,11.8464,13.1603,Kagel Mari Police Station
4,Jamestown,US,73.02,42.0970,-79.2353,Hampton Inn & Suites Jamestown
5,Bengkulu,ID,74.84,-3.8004,102.2655,Grage Hotel Bengkulu
6,Puerto Ayora,EC,78.76,-0.7393,-90.3518,Finch Bay Galapagos Hotel
9,Saint-Philippe,RE,67.59,-21.3585,55.7679,"Chambres d'hôte ""La Trinité"""
11,Luau,AO,83.16,-10.7073,22.2247,Casa- Bernardo Cândido
12,Touros,BR,86.90,-5.1989,-35.4608,INN NEW HORIZON
13,Kruisfontein,ZA,58.89,-34.0033,24.7314,Oyster Bay House Rental


In [49]:
# Add a heatmap of temperature for the vacation spots and marker for each city.
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,
             dissipating=False, max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)
# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

In [50]:
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

In [51]:
# Store the DataFrame Row.
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

In [52]:
# Add a heatmap of temperature for the vacation spots and a pop-up marker for each city.
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,dissipating=False,
             max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)

# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))